In [1]:
%reload_ext autoreload

import sys
sys.path.append('../')

import pandas as pd

from utils.data_preparation import *

In [2]:
data_path_ch = '../../../01_Data/datasets/birds_ch_2018-2022.csv'  # Provide data path of swiss dataset
data_path_de =  '../../../01_Data/datasets/birds_de_2018-2022.csv'  # Provide data path of german dataset
target = '../../../01_Data/datasets/master_bird_data_quick.csv'  # Provide data path where merged dataset shall be saved

path_translator_ids = '../../../01_Data/translators/translation_species_id_de_vs_ch.csv'
path_translator_names = '../../../01_Data/translators/translation_species_names_de_vs_ch.csv'
path_eea_grids = '../../../01_Data/eea_gridfiles/eea_europe_grids_50km/inspire_compatible_grid_50km.shp'

# Standardize raw data

In [3]:
data_de = pd.read_csv(data_path_de, delimiter=get_delimiter(data_path_de), low_memory=False)

In [ ]:
data_de.drop(columns=['estimation_code'], inplace=True)
data_de = standardize_column_names(data_de)
data_de = standardize_dtypes(data_de)
data_de = standardize_date_format(data_de, format='%d.%m.%Y')
data_de = standardize_precisions(data_de)
data_de = standardize_id_species(data_de, path_translator_ids)

In [11]:
data_ch = pd.read_csv(data_path_ch, delimiter=get_delimiter(data_path_ch), low_memory=False)

In [ ]:
data_ch = standardize_column_names(data_ch)
data_ch = standardize_dtypes(data_ch)
data_ch = standardize_precisions(data_ch)
data_ch = standardize_name_species(data_ch, path_translator_names)

In [9]:
data_de['country'] = 'de'
data_ch['country'] = 'ch'

In [10]:
master_data = pd.concat([data_de, data_ch])
master_data.to_csv(target)

# Filter for selected 27 species

In [13]:
data_path_selected_species = '../../../01_Data/archive/selected_species_of_interest.csv'  # Provide data path of species to be selected
species = pd.read_csv(data_path_selected_species, usecols = ['ornithoid','namedt', 'finale Auswahl'])
selected_species = species[species['finale Auswahl']==1]
master_selected_species = master_data[master_data.id_species.isin(selected_species.ornithoid)]

# Assign EEA grids

In [ ]:
master_selected_species = assign_eea_grids(master_selected_species, path_eea_grids)

In [ ]:
master_selected_species.to_csv('../../../01_Data/datasets/bird_data_selected_species_with_50km_grids_quick.csv')